# Iterative Prompt Development

In this lesson, you'll iteratively analyze and refine your prompts to generate marketing copy from a product fact sheet.

## Setup

In [2]:
from util import local_settings
from env_colors import TerminalTextColor as ttc
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0, messages=None):
    if not messages:
        messages = [{"role": "user", "content": prompt}]

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return completion.choices[0].message.content

sk-Yuu6ZbvcrmJ6aYp5IUQiT3BlbkFJMcylU
First LLM API example
✅ OpenAI Key loaded (sk-Yuu6ZbvcrmJ6aYp5IUQiT3BlbkFJMcylU...)


## Generate a marketing product description from a product fact sheet

### Fact Sheet - Chair SWC

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture,
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100)
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black,
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities:
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

### Prompt 1

In [4]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our stunning mid-century inspired office chair, the SWC-100. This chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. With its sleek design and multiple customization options, it is the perfect addition to any home or business setting.

The SWC-100 comes in several options of shell color and base finishes, allowing you to choose the perfect combination to match your style. You can opt for plastic back and front upholstery or go for the luxurious full upholstery in a choice of 10 fabric and 6 leather options. The base finish options include stainless steel, matte black, gloss white, or chrome.

This chair is available with or without armrests, giving you the flexibility to choose the configuration that suits your needs. It is also designed for easy raise/lower action with its pneumatic chair adjust feature.

With its 5-wheel plastic coated aluminum base, the SWC-100 offers stability and mobility. 

### 🔴 <font color="Red"> Issue 1: The text is too long</font>


👉 Limit the number of words/sentences/characters.

For example:
- "Use at most 50 words"
- "Use at most 3 sentences"
- "Use at most 280 characters"

### Prompt 2 - Limit the #words

In [5]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)
print(f"------------end------------\nCharacters: {len(response)}")
print(f"Words: {len(response.split(' '))}")

Introducing our mid-century inspired office chair, part of a beautiful furniture collection. With various color and finish options, choose between plastic or full upholstery in fabric or leather. The chair features a sturdy base, adjustable height, and is suitable for both home and business settings. Made in Italy.
------------end------------
Characters: 316
Words: 48


###  🔴 <font color="Red"> Issue 2. Text focuses on the wrong details</font>
- Ask it to focus on the aspects that are relevant to the intended audience.


```text
The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.
```

In [6]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, perfect for both home and business settings. Constructed with a durable cast aluminum shell and a 5-wheel plastic coated aluminum base, this chair offers easy raise/lower action with its pneumatic chair adjust. Choose from a variety of shell colors, base finishes, and upholstery options. Made in Italy.


#### Add the product names.

In [9]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

Use at most 50 words.

Output:

1. Description

2. Product name and country

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

1. Description:
Introducing our mid-century inspired office chair, part of a beautiful furniture collection. This chair offers a range of options, including different shell colors and base finishes. Choose between plastic or full upholstery in various fabric and leather options. The chair features a 5-wheel plastic coated aluminum base and a pneumatic chair adjust for easy height adjustment. With its stylish design and durable construction, this chair is suitable for both home and business settings. Made in Italy.

2. Product name and country:
Mid-Century Office Chair - Italy

Product IDs:
- SWC-100
- SWC-110


### 🔴 <font color="red"> Issue 3. Description needs a table of dimensions</font>

Ask it to extract information and organize it in a table.

In [16]:
prompt = f"""
Your task is to help a marketing team create a
description for a retail website of a product based
on a technical fact sheet.

Write a product description based on the information
provided in the technical specifications delimited by
triple backticks.

The description is intended for furniture retailers,
so should be technical in nature and focus on the
materials the product is constructed from.

At the end of the description, include every 7-character
Product ID in the technical specification.

After the description, include a table that gives the
product's dimensions. The table should have two columns.
In the first column include the name of the dimension.
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website.
Place the description in a <div> element.

Add an image of the chair with 100 pixels of width. The address of the image is "./img/swc-100.png"


Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
  <h2>Product Description</h2>
  <p>
    The SWC-100 Chair is a stunning addition to any office or home setting. Part of a beautiful family of mid-century inspired office furniture, this chair offers a sleek and modern design that is sure to impress. With options for shell color and base finishes, you can customize the chair to perfectly match your space.
  </p>
  <p>
    The SWC-100 Chair is available with plastic back and front upholstery or full upholstery in a variety of fabric and leather options. The base finish options include stainless steel, matte black, gloss white, or chrome. You can also choose whether to have armrests or not, depending on your preference.
  </p>
  <p>
    Constructed with a 5-wheel plastic coated aluminum base, this chair offers stability and durability. The pneumatic chair adjust allows for easy raise and lower action, ensuring optimal comfort for extended periods of sitting.
  </p>
  <p>
    The SWC-100 Chair is suitable for both home and business 

### See result in a HTML 

In [17]:
from IPython.display import display, HTML
display(HTML(response))

Dimension,Measurement (inches)
Width,"20.87"""
Depth,"20.08"""
Height,"31.50"""
Seat Height,"17.32"""
Seat Depth,"16.14"""
